In [1]:
import ipywidgets as widgets
import pandas
import qgrid
import re

from GenDBScraper.PseudomonasDotComScraper import PseudomonasDotComScraper, pdc_query

In [2]:
from GenDBScraper.StringDBScraper import StringDBScraper, stringdb_query

# The pseudomonas-fluorescens SBW25 knowledge base 

## OpenKnowledgeMap
Reference: OpenKnowldegeMaps.org - Overview of research on sbw25. Retrieved from https://openknowledgemaps.org/map/2b4a41a5661a99a129d60b6583444e67. 

In [3]:
okm_base_url = "https://https://openknowledgemaps.org/"

In [4]:
from IPython.display import IFrame
IFrame(src="https://openknowledgemaps.org/map/4aafb7d70516de0f56190d374bf398c8&embed=true", width=1000, height=1000)

## Data from pseudomonas.com 

### Select strain and gene

In [5]:
gene_widget = widgets.Text(description="Enter gene ID", value=None)

In [6]:
strain_widget = widgets.Text(description="Enter strain", value=None)

In [7]:

def on_button_clicked(b):
    pdc = PseudomonasDotComScraper(query=pdc_query(strain=strain_widget.value, feature=gene_widget.value))

    query_string = "__".join([pdc.query[0].strain, pdc.query[0].feature])
    
    pdc.connect()
    
    results = pdc.run_query()
    results = results[query_string]
    panels=list(results.keys())

    tabs = widgets.Tab()
    tabs.children = [qgrid.show_grid(results[panel], show_toolbar=False) for panel in panels if isinstance(results[panel], pandas.DataFrame)]
    for i,panel in enumerate(panels):
        tabs.set_title(i, panel)
    
    display(tabs)

In [8]:
pdc_connect_widget = widgets.Button(description="Run query on pseudomonas.com",
                                    disabled=(strain_widget.value is None) or (gene_widget.value is None))


pdc_connect_widget.on_click(on_button_clicked)

In [9]:
 display(widgets.VBox([strain_widget, gene_widget]))

In [10]:
display(pdc_connect_widget)

Button(description='Run query on pseudomonas.com', style=ButtonStyle())

2019-04-16 15:09:16,907 INFO: Connected to https://www.pseudomonas.com.
2019-04-16 15:09:17,682 INFO: Connected to https://www.pseudomonas.com/primarySequenceFeature/list?c1=name&v1=pflu0001&e1=1&term1=sbw25&assembly=complete.
2019-04-16 15:09:18,487 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1458068&view=overview.
2019-04-16 15:09:19,272 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1458068&view=overview.
2019-04-16 15:09:19,333 WARNING: No data found for Subcellular localization. Will return empty pandas.DataFrame.
2019-04-16 15:09:20,896 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1458068&view=sequence.
2019-04-16 15:09:21,667 INFO: Connected to https://www.pseudomonas.com/feature/show?id=1458068&view=functions.
2019-04-16 15:09:21,733 WARNING: No data found for Functional Classifications Manually Assigned by PseudoCAP. Will return empty pandas.DataFrame.
2019-04-16 15:09:22,467 INFO: Connected to https://www.pseudomonas.com/fe

## Data from string-db.org 

In [11]:
def run_stdb(b):
    gene_sub_pattern = re.compile(r'([a-z](?=[0-9]))')
    gene=gene_sub_pattern.sub(r'\1_', gene_widget.value)
    
    stdb = StringDBScraper(query=stringdb_query(taxonId=216595, features=[gene]))

    stdb.connect()

    stdb.update_features()

    stdb_results = dict()

    stdb_results['Network Image'] = stdb.network_image()
    stdb_results['Network Interactions'] = stdb.network_interactions()
    stdb_results['Interaction Partners'] = stdb.interaction_partners(required_score=300)
    stdb_results['Functional Enrichments'] = stdb.functional_enrichments()
    stdb_results['Interaction Enrichments'] = stdb.interaction_enrichments()
    
    with open(stdb_results['Network Image'], 'rb') as fp:
        image_widget = widgets.Image(value=fp.read(), format='svg')

    children=[qgrid.show_grid(stdb_results[k], show_toolbar=False) for k in stdb_results.keys() if not k == 'Network Image']

    children.insert(0, image_widget)

    stdb_tabs = widgets.Tab(children=children)

    for i,k in enumerate(stdb_results.keys()):
        stdb_tabs.set_title(i, k)
        
    display(stdb_tabs)

In [12]:
stdb_connect_widget = widgets.Button(description="Run query on string-db.org") 
stdb_connect_widget.on_click(run_stdb)

In [13]:
display(stdb_connect_widget)

Button(description='Run query on string-db.org', style=ButtonStyle())

2019-04-16 15:09:31,240 INFO: Connected to http://string-db.org.
2019-04-16 15:09:31,462 INFO: Connected to http://string-db.org/api/json/get_string_ids.
2019-04-16 15:09:32,181 INFO: Connected to http://string-db.org/api/image/network.
2019-04-16 15:09:32,365 INFO: Connected to http://string-db.org/api/json/network.
2019-04-16 15:09:32,614 INFO: Connected to http://string-db.org/api/json/interaction_partners.
2019-04-16 15:09:32,896 INFO: Connected to http://string-db.org/api/json/enrichment.
2019-04-16 15:09:33,279 INFO: Connected to http://string-db.org/api/json/ppi_enrichment.
